In [52]:
'''
import torch
import torch.nn as nn
import torch.quantization
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
'''

'\nimport torch\nimport torch.nn as nn\nimport torch.quantization\nimport torch.optim as optim\nfrom sklearn.datasets import load_iris\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.metrics import accuracy_score\nimport numpy as np\n'

In [53]:
#!pip install torch scikit-learn


In [54]:
'''
# 1. Load and preprocess the dataset
iris = load_iris()
X = iris.data  # shape (150, 4)
y = iris.target  # 3 classes

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to PyTorch tensors
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_data = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)'''

'\n# 1. Load and preprocess the dataset\niris = load_iris()\nX = iris.data  # shape (150, 4)\ny = iris.target  # 3 classes\n\nscaler = StandardScaler()\nX = scaler.fit_transform(X)\n\n# Convert to PyTorch tensors\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)\n\nX_train_tensor = torch.tensor(X_train, dtype=torch.float32)\ny_train_tensor = torch.tensor(y_train, dtype=torch.long)\n\nX_test_tensor = torch.tensor(X_test, dtype=torch.float32)\ny_test_tensor = torch.tensor(y_test, dtype=torch.long)\n\ntrain_data = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)\ntrain_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)'

#🧠 Model + Training


In [55]:
'''
# 2. Define a small model
class TinyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 10)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(10, 3)

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

# 3. Train the model
model_fp32 = TinyModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_fp32.parameters(), lr=0.01)

model_fp32.train()
for epoch in range(50):
    for xb, yb in train_loader:
        optimizer.zero_grad()
        out = model_fp32(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()

# Evaluate original model
model_fp32.eval()
with torch.no_grad():
    pred = model_fp32(X_test_tensor).argmax(dim=1)
    acc = accuracy_score(y_test_tensor.numpy(), pred.numpy())
print("Original Model Accuracy:", acc)
'''

'\n# 2. Define a small model\nclass TinyModel(nn.Module):\n    def __init__(self):\n        super().__init__()\n        self.fc1 = nn.Linear(4, 10)\n        self.relu = nn.ReLU()\n        self.fc2 = nn.Linear(10, 3)\n    \n    def forward(self, x):\n        return self.fc2(self.relu(self.fc1(x)))\n\n# 3. Train the model\nmodel_fp32 = TinyModel()\ncriterion = nn.CrossEntropyLoss()\noptimizer = optim.Adam(model_fp32.parameters(), lr=0.01)\n\nmodel_fp32.train()\nfor epoch in range(50):\n    for xb, yb in train_loader:\n        optimizer.zero_grad()\n        out = model_fp32(xb)\n        loss = criterion(out, yb)\n        loss.backward()\n        optimizer.step()\n\n# Evaluate original model\nmodel_fp32.eval()\nwith torch.no_grad():\n    pred = model_fp32(X_test_tensor).argmax(dim=1)\n    acc = accuracy_score(y_test_tensor.numpy(), pred.numpy())\nprint("Original Model Accuracy:", acc)\n'

#⚙️ Post-Training Quantization

In [56]:
'''# 4. Attach quantization config
model_fp32.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# 5. Prepare
model_prepared = torch.quantization.prepare(model_fp32)

# ⬅️ Calibration step
with torch.no_grad():
    for xb, _ in train_loader:
        model_prepared(xb)

# 6. Convert
model_int8 = torch.quantization.convert(model_prepared)

# 7. Evaluate
with torch.no_grad():
    pred_q = model_int8(X_test_tensor).argmax(dim=1)
    acc_q = accuracy_score(y_test_tensor.numpy(), pred_q.numpy())
print("Quantized Model Accuracy:", acc_q)

# 8. Inference
x = X_test_tensor[0].unsqueeze(0)
with torch.no_grad():
    y = model_int8(x)
print("Example Inference Output (Quantized):", y)'''


'# 4. Attach quantization config\nmodel_fp32.qconfig = torch.quantization.get_default_qconfig(\'fbgemm\')\n\n# 5. Prepare\nmodel_prepared = torch.quantization.prepare(model_fp32)\n\n# ⬅️ Calibration step\nwith torch.no_grad():\n    for xb, _ in train_loader:\n        model_prepared(xb)\n\n# 6. Convert\nmodel_int8 = torch.quantization.convert(model_prepared)\n\n# 7. Evaluate\nwith torch.no_grad():\n    pred_q = model_int8(X_test_tensor).argmax(dim=1)\n    acc_q = accuracy_score(y_test_tensor.numpy(), pred_q.numpy())\nprint("Quantized Model Accuracy:", acc_q)\n\n# 8. Inference\nx = X_test_tensor[0].unsqueeze(0)\nwith torch.no_grad():\n    y = model_int8(x)\nprint("Example Inference Output (Quantized):", y)'

#Parameter Quantization
- network pruning concept

In [57]:
'''
# Define a small model
class TinyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(10, 5)

    def forward(self, x):
        return self.fc(x)

# 1. Create and train a dummy model
model_fp32 = TinyModel()
model_fp32.eval()

# 2. Fuse layers (optional, more useful in conv models)
# 3. Attach quantization config
model_fp32.qconfig = torch.quantization.get_default_qconfig('fbgemm')  # for x86 CPUs

# 4. Prepare and convert
model_prepared = torch.quantization.prepare(model_fp32)
model_int8 = torch.quantization.convert(model_prepared)

# 5. Run inference
x = torch.randn(1, 10)
with torch.no_grad():
    y = model_int8(x)

print("Output from quantized model:", y)
'''

'\n# Define a small model\nclass TinyModel(nn.Module):\n    def __init__(self):\n        super().__init__()\n        self.fc = nn.Linear(10, 5)\n    \n    def forward(self, x):\n        return self.fc(x)\n\n# 1. Create and train a dummy model\nmodel_fp32 = TinyModel()\nmodel_fp32.eval()\n\n# 2. Fuse layers (optional, more useful in conv models)\n# 3. Attach quantization config\nmodel_fp32.qconfig = torch.quantization.get_default_qconfig(\'fbgemm\')  # for x86 CPUs\n\n# 4. Prepare and convert\nmodel_prepared = torch.quantization.prepare(model_fp32)\nmodel_int8 = torch.quantization.convert(model_prepared)\n\n# 5. Run inference\nx = torch.randn(1, 10)\nwith torch.no_grad():\n    y = model_int8(x)\n\nprint("Output from quantized model:", y)\n'

#importing

In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

#load dataset

In [59]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
akhi = pd.read_csv(url, sep=';')
akhi.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


#target variables & pre- processing

In [60]:
X = akhi.drop('quality', axis=1).values
y = akhi['quality'].values

#making the data classification friendly

y = np.where(y <= 5, 0, np.where(y == 6, 1, 2))#low, medium, high quality

# pre-processing
scaler = StandardScaler()
X = scaler.fit_transform(X)



In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_test,dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_data = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)



#Define Model

In [62]:
class WineModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(11, 64)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(64, 3)

  def forward(self, x):
    return self.fc2(self.relu(self.fc1(x)))



#Train original FP32 model

In [63]:
model_fp32 = WineModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_fp32.parameters(), lr=0.01)

model_fp32.train()
for epoch in range():
  for xb, yb in train_loader:
    optimizer.zero_grad()
    out = model_fp32(xb)
    loss = criterion(out, yb)
    loss.backward()
    optimizer.step()




#Evaluating fp32

In [64]:
model_fp32.eval()
with torch.no_grad():
  pred = model_fp32(X_test_tensor).argmax(dim=1)
  acc = accuracy_score(y_test_tensor.numpy(), pred.numpy())
print("Original Model Accuracy:", acc)

Original Model Accuracy: 0.63125


#Post-training quantization

In [65]:
torch.backends.quantized.engine = 'fbgemm'
model_fp32.qconfig = torch.quantization.get_default_qconfig('fbgemm')
model_prepared = torch.quantization.prepare(model_fp32)

with torch.no_grad():
  for xb, _ in train_loader:
    model_prepared(xb)

model_int8 = torch.quantization.convert(model_prepared)

/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


#Evaluate the quantized model

In [68]:
quantized_model = torch.quantization.quantize_dynamic(
    model_fp32, {nn.Linear}, dtype=torch.qint8
)

quantized_model.eval()
with torch.no_grad():
  pred_q = quantized_model(X_test_tensor).argmax(dim=1)
  acc_q = accuracy_score(y_test_tensor.numpy(), pred_q.numpy())
print("Quantized Model Accuracy:", acc_q)


Quantized Model Accuracy: 0.6354166666666666


/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
